In [1]:
import pandas as pd
import numpy as np

# import file
df = pd.read_csv('../../res/tp2/german_credit.csv', sep=',') # columns: admit, gre, gpa, rank; 399 rows

continuous_attrs = {'Duration of Credit (month)': ['<= 12', '<= 24', '<= 36', '<= 48', '<= 60', '> 60'],
                    'Credit Amount': ['<= 500', '<= 1000', '<= 1500', '<= 2000', '<= 2500', '<= 3000', '<= 3500', '<= 4000', 
                                      '<= 4500', '<= 5000', '<= 5500', '<= 6000', '<= 6500', '<= 7000', '<= 7500', '<= 8000',
                                      '<= 9000', '<= 10000', '<= 11000', '<= 12000', '<= 13000', '<= 14000', '<= 15000', '> 15000'],
                    'Age (years)': ['< 30', '< 40', '< 50', '< 60', '>= 60']}

In [2]:
from sklearn.model_selection import train_test_split

# Separar los datos en datos de entrenamiento y testeo
train, test = train_test_split(df, test_size=0.2)

test.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
340,1,4,30,4,3,3077,5,5,3,3,...,2,3,40,3,2,2,3,2,2,1
735,1,4,6,2,4,1338,3,3,1,1,...,4,1,62,3,2,1,3,1,1,1
247,1,4,24,3,2,4151,2,3,2,3,...,3,2,35,3,2,2,3,1,1,1
831,0,4,36,3,3,4463,1,3,4,3,...,2,3,26,3,2,2,4,1,2,1
352,1,4,48,4,9,7629,5,5,4,1,...,2,3,46,1,2,2,4,2,1,1


In [13]:
import math

positive_creditability = train.Creditability == 1

def get_entropy_term(p):
    return - p * math.log2(p) if not math.isclose(p, 0) else 0

entropy = None

def get_entropy(data, cond): # H(S)
    global entropy
    if entropy is not None:
        return entropy
    
    positive_freq = len(data[cond]) / len(data)
    negative_freq = 1 - positive_freq
    entropy = get_entropy_term(positive_freq) + get_entropy_term(negative_freq)
    
    return entropy

def get_attr_entropy(data, cond): # H(Sv)
    positive_freq = len(data[cond]) / len(data)
    negative_freq = 1 - positive_freq
    return get_entropy_term(positive_freq) + get_entropy_term(negative_freq)

def get_rows_by_attr_cond(data, attr, cond, cond_index):
    rows = None
    
    if cond_index == 0:
        rows = data.query(f'`{attr}` {cond}')
    else:
        prev_cond = continuous_attrs[attr][cond_index-1]
        rows = data.query(f'not(`{attr}` {prev_cond}) and `{attr}` {cond}')
        
    return rows
    
def gain(data, attr, class_cond):
    ret = get_entropy(train, positive_creditability)
    
    if attr in continuous_attrs:
        for i, cond in enumerate(continuous_attrs[attr]):
            rows = get_rows_by_attr_cond(data, attr, cond, i)
            ret -= len(rows) / len(data) * get_attr_entropy(rows, class_cond)
    else:
        for value in df[attr].unique():
            query = f'`{attr}` == {value}'
            print(query)
            rows = data.query(query)
            ret -= len(rows) / len(data) * get_attr_entropy(rows, class_cond)
    
    return ret

gains = {}
for attr in train.columns:
    gains[attr] = gain(train, attr, positive_creditability)
    
#df['Credit Amount'].hist(grid=True, bins=30, rwidth=0.9, color='#607c8e')
#print(get_entropy(train, train.Creditability == 1))

`Creditability` == 1
`Creditability` == 0
`Account Balance` == 1
`Account Balance` == 2
`Account Balance` == 4
`Account Balance` == 3
`Payment Status of Previous Credit` == 4
`Payment Status of Previous Credit` == 2
`Payment Status of Previous Credit` == 3
`Payment Status of Previous Credit` == 0
`Payment Status of Previous Credit` == 1
`Purpose` == 2
`Purpose` == 0
`Purpose` == 9
`Purpose` == 3
`Purpose` == 1
`Purpose` == 10
`Purpose` == 5
`Purpose` == 4
`Purpose` == 6
`Purpose` == 8
`Value Savings/Stocks` == 1
`Value Savings/Stocks` == 2
`Value Savings/Stocks` == 3
`Value Savings/Stocks` == 5


<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boo

`Value Savings/Stocks` == 4
`Length of current employment` == 2
`Length of current employment` == 3
`Length of current employment` == 4
`Length of current employment` == 1
`Length of current employment` == 5
`Instalment per cent` == 4
`Instalment per cent` == 2
`Instalment per cent` == 3
`Instalment per cent` == 1
`Sex & Marital Status` == 2
`Sex & Marital Status` == 3
`Sex & Marital Status` == 4
`Sex & Marital Status` == 1
`Guarantors` == 1
`Guarantors` == 3
`Guarantors` == 2
`Duration in Current address` == 4
`Duration in Current address` == 2
`Duration in Current address` == 3
`Duration in Current address` == 1
`Most valuable available asset` == 2
`Most valuable available asset` == 1
`Most valuable available asset` == 3
`Most valuable available asset` == 4
`Concurrent Credits` == 3
`Concurrent Credits` == 1
`Concurrent Credits` == 2
`Type of apartment` == 1
`Type of apartment` == 2
`Type of apartment` == 3
`No of Credits at this Bank` == 1
`No of Credits at this Bank` == 2
`No of Cr

<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  positive_freq = len(data[cond]) / len(data)
<ipython-input-13-742f4fe495ec>:22: UserWarning: Boo